## Cosine Similiarity계산 후 유사한 k개 JD뽑기
- 가상 JD를 넣으면 inference결과로 embedding vector 생성      **(완료)**  
- 다른 JD 와의 column별, 전체 유사도 계산 결과 생성            **(완료)**  
- 생성 결과를 추가로 계산하여 가장 유사한 jd 뽑아냄             **```(진행중)```**  
-> ```계산방법 논의 필요```

In [1]:
# Modules About Hydra
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

# Modules About Torch, Numpy
import numpy as np
import torch
import torch.nn.functional as F
import torchmetrics
import torchvision
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
from torchvision import datasets, transforms

# Modules About Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, LightningDataModule
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, ProgressBar
from lightning.pytorch.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS, STEP_OUTPUT

# Modules About Hugging Face Transformers
from transformers import AutoTokenizer, AutoModel, BertForMaskedLM, Trainer

# Modules About Language Pre-processing
import re
from konlpy.tag import Mecab

# Modules About Pandas, Matplotlib, Numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Others
from PIL import Image
from typing import List, Any
import sys
import traceback
import yaml
import ruamel.yaml
import wandb
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

/opt/homebrew/anaconda3/envs/bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
Input
data: {'input_ids': [tensor1, tensor2, ...], 'token_type_ids': [tensor1, tensor2, ...], 'attention_mask': [tensor1, tensor2, ...]}
'''


class CustomDataset(Dataset):
    def __init__(self, data) -> None:
        super().__init__()

        self.data = data
        self.keys = list(data.keys())

    def __len__(self):
        return len(self.data[self.keys[0]])

    def __getitem__(self, index) -> Any:
        item = []
        for key in self.keys:
            item.append(self.data[key][index])

        return item


In [3]:
class HFBertDataModule(pl.LightningDataModule):
    def __init__(
        self,
        tokenizer,
        train_data_dir=None,
        pred_data_dir=None,
        max_batch_size=64,
        predict_target_cols=[],
        train_target_cols=[],
        max_length=None,
        sliding_window_interval=200,
        train_test_ratio=0.9,
        train_val_ratio=0.8,
        masked_token_ratio=0.15
    ) -> None:
        super().__init__()
        self.predict_target_cols = predict_target_cols
        self.train_target_cols = train_target_cols
        self.train_data_dir = train_data_dir
        self.pred_data_dir = pred_data_dir

        self.train_test_ratio = train_test_ratio
        self.train_val_ratio = train_val_ratio

        self.batch_size = max_batch_size
        if predict_target_cols:
            self.predict_batch_size = int(
                max_batch_size / len(predict_target_cols)) * len(predict_target_cols)

        # load Bert Tokenizer
        self.tokenizer = tokenizer

        if max_length:
            self.max_length = max_length
        else:
            self.max_length = tokenizer.model_max_length

        self.sliding_window_interval = sliding_window_interval

        self.masked_token_ratio = masked_token_ratio

    def prepare_data(self) -> None:
        # load predict data
        try:
            self.predict_data_pd = pd.read_csv(
                self.pred_data_dir)
        except:
            print('No inference data available!')
            self.predict_data_pd = None

        if self.predict_data_pd is not None and self.predict_target_cols:
            # serialize columns
            predict_data_serialized = []
            for row in range(len(self.predict_data_pd)):
                for col in self.predict_target_cols:
                    predict_data_serialized.append(
                        self.predict_data_pd.iloc[row][col])

            # make tokens
            self.predict_tokens = self.tokenizer(
                predict_data_serialized, return_tensors='pt', padding=True, truncation=True)

            # make predict dataset
            self.predict_dataset = CustomDataset(self.predict_tokens)
            self.predict_token_keys = self.predict_tokens.keys()

    def setup(self, stage: str) -> None:
        # load train data
        try:
            self.train_data_pd = pd.read_csv(
                self.train_data_dir)
        except:
            print('No training data available!')
            self.train_data_pd = None

        if self.train_data_pd is not None and self.train_target_cols:
            # serialize columns
            train_data_serialized = []
            for col in self.train_target_cols:
                train_data_serialized += list(self.train_data_pd[col])

            # make tokens
            self.train_tokens = self.tokenizer(
                train_data_serialized, return_tensors='pt', padding=True)
            self.train_token_keys = self.train_tokens.keys()

            # slicing tokens by a sliding window
            current_token_length = self.train_tokens['input_ids'].shape[1]
            if current_token_length > self.max_length:
                self.train_tokens_sliced = self._make_sliced_tokens(
                    self.train_tokens, current_token_length)
            else:
                self.train_tokens_sliced = self.train_tokens

            # make train dataset
            train_dataset = CustomDataset(self.train_tokens_sliced)

            # split train val test datasets
            self.train_dataset, self.val_dataset, self.test_dataset = random_split(
                train_dataset,
                [
                    self.train_test_ratio * self.train_val_ratio,
                    self.train_test_ratio * (1 - self.train_val_ratio),
                    1 - self.train_test_ratio
                ]
            )

    def _collate_fn_predict(self, batch):
        '''
        Inputs
        batch: [[tensor1_1, tensor1_2, tensor1_3], [tensor2_1, tensor2_2, tensor2_3], ...]
        self.predict_token_keys: ['input_ids', 'token_type_ids', 'attention_mask']

        Output
        dict_by_keys: {'input_ids': [tensor1_1, tensor2_1, ...], 'token_type_ids': [tensor1_2, tensor2_2, ...], 'attention_mask': [tensor1_3, tensor2_3, ...]}
        '''
        list_by_keys = list(zip(*batch))
        dict_by_keys = {}
        for i, key in enumerate(self.predict_token_keys):
            dict_by_keys[key] = torch.stack(list_by_keys[i])

        return dict_by_keys

    def _collate_fn_train(self, batch):
        list_by_keys = list(zip(*batch))
        dict_by_keys = {}
        for i, key in enumerate(self.train_token_keys):
            dict_by_keys[key] = torch.stack(list_by_keys[i])

        dict_by_keys['labels'] = dict_by_keys['input_ids'].clone()

        for i, tokens in enumerate(dict_by_keys['input_ids']):
            self._make_tokens(
                tokens, dict_by_keys['labels'][i], self.masked_token_ratio)

        return dict_by_keys

    def _make_sliced_tokens(self, tokens, tokens_length):
        train_tokens_sliced = {}
        for key in self.train_token_keys:
            train_tokens_sliced[key] = []

        for i in range(len(tokens[key])):
            window_index = 0
            while True:
                if window_index + self.max_length <= tokens_length:
                    for key in self.train_token_keys:
                        train_tokens_sliced[key].append(
                            tokens[key][i][window_index:window_index + self.max_length])

                    if tokens[key][i][window_index + self.max_length - 1] != self.tokenizer.pad_token_id:
                        window_index += self.sliding_window_interval
                        continue
                break

        return train_tokens_sliced

    def _make_tokens(self, tensor1, tensor2, mask_token_ratio, masking_ratio=[0.8, 0.1, 0.1]):
        assert sum(masking_ratio) == 1

        token_len = 0
        for token in tensor1:
            if token != self.tokenizer.pad_token_id:
                token_len += 1
                continue
            break

        masked_tokens = torch.tensor(np.random.choice(range(
            1, token_len - 1), int((token_len - 2) * mask_token_ratio)), dtype=torch.int)
        token_types = torch.randint_like(masked_tokens, 1, 101)

        tensor2[:] = -100
        tensor2[masked_tokens] = tensor1[masked_tokens]
        tensor1[masked_tokens] = torch.where(token_types <= int(masking_ratio[0] * 100), self.tokenizer.mask_token_id, torch.where(token_types <= int((1 - masking_ratio[2]) * 100), torch.randint(0, self.tokenizer.vocab_size, (len(masked_tokens),)), tensor1[masked_tokens]))

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(self.train_dataset, batch_size=self.batch_size, collate_fn=self._collate_fn_train)

    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.val_dataset, batch_size=self.batch_size, collate_fn=self._collate_fn_train)

    def test_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.test_dataset, batch_size=self.batch_size, collate_fn=self._collate_fn_train)

    def predict_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.predict_dataset, batch_size=self.predict_batch_size, collate_fn=self._collate_fn_predict)

In [4]:
class HFBertTask(pl.LightningModule):
    def __init__(self, tokenizer, optimizer=None, lr_scheduler=None, predict_model=None,
                 train_model=None, predict_target_cols=[],
                 train_target_cols=[]) -> None:
        super().__init__()
        self.predict_target_cols = predict_target_cols
        self.train_target_cols = train_target_cols

        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.tokenizer = tokenizer

        # self.save_hyperparameters(
        #     "optimizer", "tokenizer",
        #     "lr_scheduler", "predict_target_cols",
        #     "train_target_cols")

        self.predict_model = predict_model
        self.train_model = train_model
        self.training_step_outputs = []
        self.validation_step_outputs = []

        self.acc_func = None

    def training_step(self, batch, batch_idx) -> STEP_OUTPUT:
        if not (self.train_model or self.train_target_cols):
            print('No train_model or train_target_cols available!')
            return

        outputs = self.train_model(**batch)

        metrics = {
            'train_loss': outputs.loss
        }
        self.training_step_outputs.append(metrics)
        self.log_dict(metrics, prog_bar=True)

        return outputs.loss

    def validation_step(self, batch, batch_idx) -> STEP_OUTPUT | None:
        if not (self.train_model or self.train_target_cols):
            print('No train_model or train_target_cols available!')
            return

        outputs = self.train_model(**batch)

        metrics = {
            'val_loss': outputs.loss
        }
        self.validation_step_outputs.append(metrics)
        self.log_dict(metrics, prog_bar=True)

        return outputs.loss

    def on_validation_epoch_end(self):
        if not (self.training_step_outputs and self.validation_step_outputs):
            return

        train_avg_loss = torch.stack([x["train_loss"]
                                      for x in self.training_step_outputs]).mean()
        metrics = {
            "train_avg_loss": train_avg_loss
        }
        self.log_dict(metrics)

        val_avg_loss = torch.stack([x["val_loss"]
                                    for x in self.validation_step_outputs]).mean()
        metrics = {
            "val_avg_loss": val_avg_loss
        }
        self.log_dict(metrics)

        print("\n" +
              (f'Epoch {self.current_epoch}, Avg. Training Loss: {train_avg_loss:.3f} ' +
               f'Avg. Validation Loss: {val_avg_loss:.3f}'), flush=True)

        self.training_step_outputs.clear()
        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx) -> None:
        if not (self.train_model or self.train_target_cols):
            print('No train_model or train_target_cols available!')
            return

        outputs = self.train_model(**batch)

        metrics = {
            'test_loss': outputs.loss
        }
        self.log_dict(metrics, prog_bar=True)

        sentence_index, mask_token_index = (
            batch['input_ids'] == self.tokenizer.mask_token_id).nonzero(as_tuple=True)

        predicted_token_id = []
        for index, sentence in enumerate(sentence_index):
            if sentence >= len(predicted_token_id):
                predicted_token_id.append([])

            predicted_token_id[-1].append(self.tokenizer.decode(
                outputs.logits[sentence, mask_token_index[index]].argmax(axis=-1)))

        random_numbers = torch.randint(low=0, high=len(
            predicted_token_id), size=(int(len(batch['input_ids']) / 2),))

        original_token_id = self.tokenizer.batch_decode(batch['input_ids'])

        for i in random_numbers:
            print(predicted_token_id[i], original_token_id[i])

    def predict_step(self, batch, batch_idx, dataloader_idx: int = 0):
        if not (self.predict_model or self.predict_target_cols):
            print('No predict_model or predict_target_cols available!')
        outputs = self.predict_model(**batch)
        # these are the sentence embedding vectors (768 dim each)
        pooler_outputs = outputs[1]
        outputs_concated = []
        for i in range(int(len(pooler_outputs) / len(self.predict_target_cols))):
            outputs_concated.append(torch.concat(list(
                pooler_outputs[i * len(self.predict_target_cols):(i + 1) * len(self.predict_target_cols)])))
            # Concatenating sentence embedding vectors from a job description

        return torch.stack(outputs_concated)

    def configure_optimizers(self):
        optimizer = self.optimizer
        if self.lr_scheduler is not None:
            return [optimizer], [self.lr_scheduler]
        else:
            return optimizer

In [5]:
# Inference

with initialize(version_base=None, config_path="./"):
    cfg = compose(config_name="config.yaml")

predict_target_cols = cfg.universal.predict_target_cols
tokenizer = instantiate(cfg.universal.tokenizer.pred)
data_module = HFBertDataModule(
    **cfg.predict_data,
    tokenizer=tokenizer,
    predict_target_cols=predict_target_cols
)

models = [(instantiate(cfg.models[model].mlm), instantiate(cfg.models[model].pred))
          for model in dir(cfg.models)]

final_vector_list = []
for model in models:
    mlm_model = model[0]
    predict_model = model[1]
    predict_model.pooler.dense = mlm_model.cls.predictions.transform.dense
    task = HFBertTask(tokenizer=tokenizer, train_model=mlm_model,
                      predict_model=predict_model,
                      predict_target_cols=predict_target_cols)

    trainer = pl.Trainer()

    # this list contains tensors of each output of batch running
    predicted_embedding_vectors = trainer.predict(
        task, datamodule=data_module)
    concatenated_embedding_vectors = torch.concat(
        predicted_embedding_vectors, dim=-2)
    final_vector_list.append(concatenated_embedding_vectors)
final_vector_list

Some weights of the model checkpoint at ./huggingface/kcbert_large_2epoch were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./huggingface/kcbert_large_2epoch and are newly initialized: ['bert.pooler.dense.bias', 'be

No training data available!
Predicting DataLoader 0: 100%|██████████| 41/41 [00:12<00:00,  3.25it/s]


[tensor([[0.9855, 0.3175, 0.9118,  ..., 0.9905, 0.9966, 0.9900],
         [0.9750, 0.5082, 0.9775,  ..., 0.9976, 0.9941, 0.9851],
         [0.9717, 0.3061, 0.9589,  ..., 0.9412, 0.9963, 0.9707],
         ...,
         [0.6947, 0.8666, 0.6332,  ..., 0.9643, 0.9226, 0.5185],
         [0.8408, 0.8370, 0.5661,  ..., 0.9975, 0.9975, 0.9514],
         [0.9617, 0.6414, 0.9750,  ..., 0.9587, 0.9478, 0.8475]])]

In [6]:
# Inference_check : 위의 것과 똑같은지 확인하기

with initialize(version_base=None, config_path="./"):
    cfg = compose(config_name="config.yaml")

predict_target_cols = cfg.universal.predict_target_cols
tokenizer = instantiate(cfg.universal.tokenizer.pred)
data_module = HFBertDataModule(
    **cfg.predict_data,
    tokenizer=tokenizer,
    predict_target_cols=predict_target_cols
)

models = [(instantiate(cfg.models[model].mlm), instantiate(cfg.models[model].pred))
          for model in dir(cfg.models)]

final_vector_list_check = []
for model in models:
    mlm_model = model[0]
    predict_model = model[1]
    predict_model.pooler.dense = mlm_model.cls.predictions.transform.dense
    task = HFBertTask(tokenizer=tokenizer, train_model=mlm_model,
                      predict_model=predict_model,
                      predict_target_cols=predict_target_cols)

    trainer = pl.Trainer()

    # this list contains tensors of each output of batch running
    predicted_embedding_vectors = trainer.predict(
        task, datamodule=data_module)
    concatenated_embedding_vectors = torch.concat(
        predicted_embedding_vectors, dim=-2)
    final_vector_list_check.append(concatenated_embedding_vectors)
if torch.all(final_vector_list[0] == final_vector_list_check[0]):
    print("✅ predict 결과가 동일합니다!")
else :
    print("❌ predict 결과가 다릅니다!")

Some weights of the model checkpoint at ./huggingface/kcbert_large_2epoch were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./huggingface/kcbert_large_2epoch and are newly initialized: ['bert.pooler.dense.bias', 'be

No training data available!
Predicting DataLoader 0: 100%|██████████| 41/41 [00:12<00:00,  3.28it/s]
✅ predict 결과가 동일합니다!


## 생성 vector 확인

In [7]:
final_vector_list

[tensor([[0.9855, 0.3175, 0.9118,  ..., 0.9905, 0.9966, 0.9900],
         [0.9750, 0.5082, 0.9775,  ..., 0.9976, 0.9941, 0.9851],
         [0.9717, 0.3061, 0.9589,  ..., 0.9412, 0.9963, 0.9707],
         ...,
         [0.6947, 0.8666, 0.6332,  ..., 0.9643, 0.9226, 0.5185],
         [0.8408, 0.8370, 0.5661,  ..., 0.9975, 0.9975, 0.9514],
         [0.9617, 0.6414, 0.9750,  ..., 0.9587, 0.9478, 0.8475]])]

In [8]:
final_vector_list[0].shape

torch.Size([41, 5120])

In [9]:
torch.save(final_vector_list[0],'kclarge2_sample.pth')

### 다른 JD 와의 column별, 전체 유사도 계산 결과 생성


In [10]:
#가상 JD와 가장 유사한 K개의 원본 JD뽑아내기
#추가작업필요

def Top_k_JD(vec_mock: torch.tensor, origin_JD_csv, vec_origin: torch.tensor, k):
    cosine_sim = torch.nn.functional.cosine_similarity(vec_mock.unsqueeze(0),vec_origin, dim=1).argsort(descending=True, dim =0)

    return origin_JD_csv.loc[cosine_sim.tolist()[:k]]

In [11]:
#가상 JD와 column별 cosine similarity DataFrame 생성

def Cosine_Sim_Col(vec_mock: torch.tensor, vec_origin: torch.tensor):
    
    col_name = ['자격요건','우대조건','복지','회사소개','주요업무','전체']
    cosine_dic = {}

    origins = torch.chunk(vec_origin, chunks=5, dim=1)
    mocks = torch.chunk(vec_mock, chunks=5, dim=0)

    cosine_dic['전체'] = pd.Series(torch.nn.functional.cosine_similarity(vec_mock.unsqueeze(0),vec_origin, dim=1))

    for origin, mock, col in zip(origins, mocks,col_name):
        cosine_dic[col]  = pd.Series(torch.nn.functional.cosine_similarity(mock.unsqueeze(0),origin, dim=1))

    return pd.DataFrame(cosine_dic)

- 입력 데이터  
mock_JD = ```QnA를 통해 사용자에게 얻은 가상 jd의 embedding_vector data```  
origin_JD_csv = ```서버에 올라간 jd전체 data```  
origin_JD_em = ```서버에 올라간 jd전체 embedding_vector data```   
k = ```최대 k개의 데이터를 뽑음```
  
- cosine 추가 계산 방법 논의  
-> ```합계, 나누기, 평균, 가중치 ...```

In [13]:
# 예시 코드:
origin_JD_csv = pd.read_csv('../../dataset/pre_result.csv')
origin_JD_em = torch.load('kclarge2_sample.pth')
mock_JD = origin_JD_em[:][0]


#cosine 유사도 dataframe 생성
cosine_df = Cosine_Sim_Col(mock_JD, origin_JD_em)

#Top k개 공고 출력
# JD_list = Top_k_JD(mock_JD, origin_JD_csv, origin_JD_em, k)
# JD_list

In [14]:
cosine_df

,전체,자격요건,우대조건,복지,회사소개,주요업무
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.995529,0.995498,0.997423,0.997143,0.991397,0.996710
2,0.959888,0.994700,0.930170,0.996645,0.881122,0.994484
3,0.987310,0.994324,0.996085,0.994374,0.995708,0.957762
4,0.971447,0.929232,0.993530,0.997176,0.995605,0.942725
5,0.956741,0.937767,0.994977,0.897395,0.997461,0.952567
6,0.964319,0.932812,0.993328,0.897715,0.995384,0.996261
7,0.901509,0.913314,0.920894,0.869809,0.996247,0.798138
8,0.985314,0.991976,0.993962,0.997730,0.997639,0.946985
9,0.992250,0.985995,0.994934,0.989149,0.996831,0.994672
